In [1]:
import time
import polars as pl
import requests
import json
import pathlib
from typing import List, Tuple
from unipressed import IdMappingClient

In [2]:
gene_id_tsv = "../test/zea_mays_test/zea_mays_random_gene_list.tsv"
query_db = "Ensembl_Genomes"
target_db = "UniProtKB"
json_dir = "zea_mays_random_gene_afinfo"
data_url = "cifUrl" # or "pdbUrl", "bcifUrl", "paeImageUrl", "paeDocUrl"
structure_dir = "zea_mays_random_gene_mmcif"
id_mapping_all_file = "zea_mays_random_gene_idmapping_all.tsv"

In [3]:
# Parameters
gene_id_tsv = "/tmp/3yskfl25/stg8b7476b3-701f-4b7f-bd1c-29f8c2e753a3/oryza_sativa_random_100genes_list.tsv"
query_db = "Ensembl_Genomes"
target_db = "UniProtKB"
json_dir = "os_100_genes_afinfo_json"
data_url = "cifUrl"
structure_dir = "os_100_genes_mmcif"
id_mapping_all_file = "os_100_genes_idmapping_all.tsv"


&nbsp;

&nbsp;

## 1. UniProt ID mapping step

In [4]:
def chunk_list(lst: List, chunk_size: int) -> List[List]:
    """Split a gene list into chunks"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

In [5]:
def batch_id_mapping(from_db: str, to_db: str, ids: List[str], chunk_size: int = 100) -> Tuple[pl.DataFrame, List[str]]:
    """function for batch id mapping"""
    all_results = []
    all_unmapped = []
    chunked_ids = chunk_list(ids, chunk_size)

    for i, chunk in enumerate(chunked_ids):
        print(f"Processing chunk {i+1}/{len(chunked_ids)}...")
        
        # create request and run
        request = IdMappingClient.submit(source=from_db, dest=to_db, ids=chunk)
        
        # process results
        chunk_results = list(request.each_result())
        mapped_results = [{"from": item["from"], "to": item["to"]} for item in chunk_results]
        all_results.extend(mapped_results)

        # record unmapped ids
        mapped_ids = set(item["from"] for item in mapped_results)
        unmapped = [id for id in chunk if id not in mapped_ids]
        all_unmapped.extend(unmapped)

        # avoid API rate limit
        time.sleep(3)

    # convert results to DataFrame
    final_df = pl.DataFrame(all_results)
    return final_df, all_unmapped

In [6]:
gene_ids = pl.read_csv(
    gene_id_tsv,
    separator='\t'
).get_column("From").to_list()

mapped_df, unmapped_ids = batch_id_mapping(
    query_db,
    target_db,
    gene_ids
)

display(mapped_df)
display(unmapped_ids)

Processing chunk 1/1...


from,to
str,str
"""Os02g0249600""","""Q02897"""
"""Os07g0216600""","""Q7X7E6"""
"""Os02g0778400""","""Q6K7H2"""
"""Os08g0559300""","""Q6YZI0"""
"""Os01g0152300""","""Q9LGI2"""


['Os12g0269700',
 'Os10g0410900',
 'Os05g0403000',
 'Os06g0127250',
 'Os09g0349700',
 'Os03g0735150',
 'Os08g0547350',
 'Os06g0282400',
 'Os05g0576750',
 'Os10g0164500',
 'Os07g0201300',
 'Os01g0567200',
 'Os05g0563050',
 'Os03g0660050',
 'Os11g0436450',
 'Os01g0924300',
 'Os03g0388300',
 'Os10g0422566',
 'Os06g0481900',
 'Os03g0266700',
 'Os04g0542900',
 'Os08g0375700',
 'Os02g0599400',
 'Os01g0664500',
 'Os02g0695600',
 'Os03g0589300',
 'ENSRNA049441959',
 'Os03g0100800',
 'Os04g0591300',
 'Os05g0165400',
 'Os04g0607200',
 'Os11g0557800',
 'Os03g0367900',
 'Os03g0354500',
 'Os10g0495400',
 'Os10g0418900',
 'Os08g0213400',
 'Os03g0572250',
 'Os09g0249000',
 'Os03g0651201',
 'Os02g0781400',
 'Os02g0564400',
 'Os09g0500151',
 'Os11g0114900',
 'Os02g0131300',
 'Os05g0198950',
 'Os05g0157300',
 'ENSRNA049471020',
 'Os04g0509400',
 'ENSRNA049475823',
 'Os09g0552900',
 'ENSRNA049468095',
 'Os10g0543800',
 'Os09g0568400',
 'Os06g0294100',
 'Os12g0577733',
 'Os02g0816000',
 'Os03g0250000',
 '

In [7]:
mapped_df2 = mapped_df.rename(
    {
        "from": "From",
        "to": "UniProt Accession"
    }
)

display(mapped_df2)

From,UniProt Accession
str,str
"""Os02g0249600""","""Q02897"""
"""Os07g0216600""","""Q7X7E6"""
"""Os02g0778400""","""Q6K7H2"""
"""Os08g0559300""","""Q6YZI0"""
"""Os01g0152300""","""Q9LGI2"""


In [8]:
print(mapped_df.is_empty())
print(len(unmapped_ids) == 0)


False
False


&nbsp;

&nbsp;

## 2. UniProt re-mapping 

## 3. Concatenate the two dataframes

## 4. AlphaFoldDB metadata JSON files

In [9]:
def fetch_uniprot_data(ensembl_ids: List[str]) -> pl.DataFrame:
    results = []

    for id in ensembl_ids:
        print(f"Processing {id}...")
        url = (
            f"https://rest.uniprot.org/uniprotkb/search?"
            f"query=gene:{id}&format=json"
        )
        response = requests.get(url)
        
        if response.status_code == 200:
            data = json.loads(response.text)
            for item in data.get('results', []):
                primary_accession = item.get('primaryAccession', '')
                secondary_accessions = item.get('secondaryAccessions', [])
                all_accessions = [primary_accession] + secondary_accessions
                
                for accession in all_accessions:
                    entry = {
                        "From": id,
                        "UniProt Accession": accession
                    }
                    
                    # Check if the accession is a match for the gene 
                    # (e.g. Os03g0293000 matches OrderedLocusNames)
                    match_found = False
                    for gene in item.get('genes', []):
                        for locus in gene.get('orderedLocusNames', []):
                            if locus.get('value', '') == id:
                                match_found = True
                                break
                        if match_found:
                            break
                    
                    if match_found:
                        results.append(entry)
        else:
            print(f"Error fetching data for {id}: {response.status_code}")
        
        time.sleep(1)

    return pl.DataFrame(results)

In [10]:
def get_af_json(dataframe: pl.DataFrame, target_dir: str):
    """
    Get JSON file from AlphaFoldDB
    """
    pathlib.Path(target_dir).mkdir(parents=True, exist_ok=True)
    
    for row in dataframe.iter_rows():
        gene_id = row[0]
        uniprot_id = row[1]
        
        json_file_name = pathlib.Path(target_dir) / f"{gene_id}_{uniprot_id}_info.json"
        
        if json_file_name.exists():
            message_1 = f"{json_file_name} already exists"
            print(message_1)
            continue
        
        request_url = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}"
        
        try:
            response = requests.get(request_url, headers={"Accept": "application/json"}, timeout=30)
            response.raise_for_status()
            
            if response.text:
                data = json.loads(response.text) # parse json
                if isinstance(data, list) and len(data) > 0:
                    message_2 = f"AlphaFold ID {uniprot_id} found in AlphaFold"
                    print(message_2)
                    with open(json_file_name, 'w') as f:
                        json.dump(data[0], f, indent=4)
                else:
                    message_3 = f"AlphaFold ID {uniprot_id} not found in AlphaFold"
                    print(message_3)
            else:
                message_4 = f"Empty response for AlphaFold ID {uniprot_id}"
                print(message_4)
        except requests.exceptions.RequestException as e:
            message_5 = f"Request failed: {e}"
            print(message_5)
            message_6 = f"AlphaFold ID {uniprot_id} not found in AlphaFold"
            print(message_6)
        time.sleep(5)

In [11]:
unmapped_df = fetch_uniprot_data(unmapped_ids)
display(unmapped_df)

Processing Os12g0269700...


Processing Os10g0410900...


Processing Os05g0403000...


Processing Os06g0127250...


Processing Os09g0349700...


...

Processing Os03g0811600...


Processing Os01g0322300...


Processing Os07g0655300...


Processing Os01g0875300...


From,UniProt Accession
str,str
"""Os12g0269700""","""C7J9I2"""
"""Os10g0410900""","""Q0IXS1"""
"""Os10g0410900""","""A0A0P0XU49"""
"""Os05g0403000""","""Q6AUL2"""
"""Os05g0403000""","""B9FID2"""
…,…
"""Os07g0655300""","""Q8H3E5"""
"""Os07g0655300""","""Q0D414"""
"""Os07g0655300""","""B9FUL9"""


In [12]:
if len(unmapped_ids) > 0:
    unmapped_df = fetch_uniprot_data(unmapped_ids)
    display(unmapped_df)

    # Concatenate the two dataframes
    if not unmapped_df.is_empty():
        id_mapping_df = pl.concat(
            [
                mapped_df2,
                unmapped_df
            ],
            how="vertical_relaxed"
        ).sort(
            by="From",
            descending=False
        )
        display(id_mapping_df)
        # Get AlphaFold metadata JSON files
        get_af_json(id_mapping_df, json_dir)
    else:
        print("unmapped dataframe is empty, skipping get_af_json.")
        get_af_json(mapped_df2, json_dir)
else:
    print("unmapped_ids is empty, skipping fetch_uniprot_data.")
    get_af_json(mapped_df2, json_dir)

Processing Os12g0269700...


Processing Os10g0410900...


Processing Os05g0403000...


Processing Os06g0127250...


Processing Os09g0349700...


...

AlphaFold ID Q0IMJ5 found in AlphaFold


Request failed: 404 Client Error: Not Found for url: https://alphafold.ebi.ac.uk/api/prediction/A0A0P0YBZ4
AlphaFold ID A0A0P0YBZ4 not found in AlphaFold


AlphaFold ID A0A0P0YBQ9 found in AlphaFold


AlphaFold ID Q0IM90 found in AlphaFold


Request failed: 404 Client Error: Not Found for url: https://alphafold.ebi.ac.uk/api/prediction/Q2QMY1
AlphaFold ID Q2QMY1 not found in AlphaFold


&nbsp;

&nbsp;

## 5. Download CIF files

In [13]:
def get_cif_file(json_dir_path: str, output_dir_path: str):
    """
    Get CIF file from JSON files retrieved from AlphaFoldDB
    
    Args:
        json_dir_path: Directory containing AlphaFold JSON metadata files
        output_dir_path: Directory to save downloaded CIF files
    """
    pathlib.Path(output_dir_path).mkdir(parents=True, exist_ok=True)

    for json_file in pathlib.Path(json_dir_path).glob("*.json"):
        with open(json_file, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                cif_url = data.get(data_url)
                
                if not cif_url:
                    print(f"No {data_url} found in {json_file}")
                    continue
                    
                # Extract filename from URL and create output path
                cif_filename = pathlib.Path(cif_url).name
                output_file = pathlib.Path(output_dir_path) / cif_filename
                
                # Skip if file already exists
                if output_file.exists():
                    print(f"{output_file} already exists")
                    continue
                
                print(f"Downloading {cif_url}")
                response = requests.get(cif_url, timeout=30)
                response.raise_for_status()
                
                # Save CIF file
                output_file.write_bytes(response.content)
                print(f"Saved {output_file}")
                
            except json.JSONDecodeError:
                print(f"Error parsing JSON file: {json_file}")
            except requests.exceptions.RequestException as e:
                print(f"Error downloading CIF file: {e}")
            
            # Rate limiting
            time.sleep(1)

In [14]:
get_cif_file(json_dir, structure_dir)

Saved os_100_genes_mmcif/AF-Q0IM90-F1-model_v6.cif


Saved os_100_genes_mmcif/AF-A0A0P0YBQ9-F1-model_v6.cif


Saved os_100_genes_mmcif/AF-Q0IMJ5-F1-model_v6.cif


Saved os_100_genes_mmcif/AF-A0A0N7KTX9-F1-model_v6.cif


Saved os_100_genes_mmcif/AF-Q0INQ2-F1-model_v6.cif


...

Saved os_100_genes_mmcif/AF-Q657K0-F1-model_v6.cif


Saved os_100_genes_mmcif/AF-Q657N1-F1-model_v6.cif


Saved os_100_genes_mmcif/AF-Q9LGI2-F1-model_v6.cif


Saved os_100_genes_mmcif/AF-Q658C6-F1-model_v6.cif


Saved os_100_genes_mmcif/AF-Q657Z6-F1-model_v6.cif


&nbsp;

&nbsp;

## 6. Save all results

In [15]:
# Save all results
if len(unmapped_ids) > 0 and not unmapped_df.is_empty():
    id_mapping_df.write_csv(id_mapping_all_file, separator="\t")

elif unmapped_df.is_empty():
    print("re-mapping process is skipped, mapping results are saved in id_mapping_df.write_csv.")
    mapped_df2.write_csv(id_mapping_all_file, separator="\t")

else:
    print("unmapped_ids is empty, skipping re-mapping process.")
    mapped_df2.write_csv(id_mapping_all_file, separator="\t")